In [1]:
import torch
import torch.nn as nn


In [1]:
import numpy as np
mfcc_path = "/opt/ml/ytdataset/audios_mfcc/train/100_Paul Rice “Waltz for Ella” FREE SHEET MUSIC P Barton FEURICH piano.mp4.pickle"
mfcc = np.load(mfcc_path, allow_pickle=True)


In [13]:
mfcc_arr = np.asarray([mfcc[i] for i in range(1, 6)])

In [14]:
mfcc_arr.shape

(5, 13, 3)

In [7]:
import numpy as np
import time
total_time = []
for i in range(100):
    x = [np.random.rand(100, 900) for _ in range(5)]
    st = time.time()
    torch.Tensor(np.asarray(x)).float().cuda()
    ed = time.time()
    total_time.append((ed -st) * 1000)



In [11]:
total_time

[4.080057144165039,
 11.507272720336914,
 5.411624908447266,
 3.293752670288086,
 3.209352493286133,
 3.1833648681640625,
 2.3069381713867188,
 1.260995864868164,
 1.3887882232666016,
 1.8720626831054688,
 3.175020217895508,
 3.1714439392089844,
 3.262042999267578,
 2.744436264038086,
 3.1833648681640625,
 3.217935562133789,
 3.1731128692626953,
 3.1888484954833984,
 3.1807422637939453,
 3.1893253326416016,
 1.4352798461914062,
 1.3113021850585938,
 1.268148422241211,
 1.257181167602539,
 1.2521743774414062,
 1.2812614440917969,
 1.2586116790771484,
 1.253366470336914,
 3.1280517578125,
 3.1528472900390625,
 3.1349658966064453,
 3.219127655029297,
 3.1366348266601562,
 2.765655517578125,
 3.150463104248047,
 3.1380653381347656,
 3.1452178955078125,
 3.117084503173828,
 3.1538009643554688,
 3.1280517578125,
 3.118276596069336,
 3.147602081298828,
 1.0852813720703125,
 1.1088848114013672,
 1.108407974243164,
 1.1470317840576172,
 1.1477470397949219,
 2.3508071899414062,
 1.10602378845214

In [10]:
np.min(np.asarray(total_time)), np.median(np.asarray(total_time))

(1.0802745819091797, 2.7550458908081055)

In [67]:
net = nn.Sequential(
    nn.Conv2d(5, 32, 2, 1),
    nn.ReLU(),
    nn.Conv2d(32, 64, 2, 1),
    nn.ReLU(),
    nn.MaxPool2d((11, 1)),
    # nn.AdaptiveAvgPool2d((1,1))
    # nn.Conv2d(32, 64, 3, 1, 1),
    # nn.ReLU(),
    # nn.MaxPool2d(2, 2)
    nn.Flatten(start_dim=1)
)

In [68]:
x = torch.tensor(mfcc_arr).float().unsqueeze(0)
print(net(x).shape, x.shape)
# torch.nn.init.xavier_uniform_(
#     self.conv.weight, gain=torch.nn.init.calculate_gain(w_init_gain))

torch.Size([1, 64]) torch.Size([1, 5, 13, 3])


In [72]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(5, 32, 2, 1),
            nn.ReLU(),            
            nn.Conv2d(32, 64, 2, 1),
            nn.ReLU(),
            nn.MaxPool2d((11, 1)),
            nn.Flatten(start_dim=1)
        )


    def forward(self, x):
        x = self.conv_layers(x)
        return x

# 간단한 RNN 모델 정의
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out        

In [73]:
x = CNNModel()(x)

In [74]:
cnn.shape

torch.Size([1, 64])

: 

In [7]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=1764, hidden_size=256, num_layers=3, output_size=64):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # 초기 hidden state와 cell state 설정
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # LSTM 모델 적용
        out, _ = self.lstm(x, (h0, c0))
        print(out.shape)

        # 마지막 타임스텝의 hidden state를 추출하여 특징 벡터로 변환
        features = self.fc(out[:, -1, :])

        return features
    

class RESTM(nn.Module):
    def __init__(self, pretrained=False, min_key=5, max_key=80, lstm_output_size=256, resnet_output_size=256):
        super(RESTM, self).__init__()
        self.num_classes=(max_key-min_key)+1

        self.lstm = LSTMModel(output_size=lstm_output_size)
        self.resnet = ResNet(BasicBlock, layers=[3, 3, 3, 3, 2], top_channel_nums=2048, reduced_channel_nums=256, num_classes=resnet_output_size)
        if pretrained:
            model_path = '/opt/ml/Audeo/Audeo_github/Video2Roll_models/Video2RollNet.pth' # change to your path
            checkpoint = torch.load(model_path, map_location="cuda:0")
            checkpoint['fc.weight'] = self.resnet.fc.weight
            checkpoint['fc.bias'] = self.resnet.fc.bias
            self.resnet.cuda()
            self.resnet.load_state_dict(checkpoint)

        self.fc = nn.Sequential(
            nn.Linear(lstm_output_size, 256),
            nn.BatchNorm1d(256),
            nn.Dropout(),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.Dropout(),
            nn.ReLU(True),
            nn.Linear(512, self.num_classes))
    
    def forward(self, x_frames, x_audio):
        x_frames = self.resnet(x_frames)
        x_audio = self.lstm(x_audio)
        # x_frames_audio = torch.concatenate((x_frames, x_audio), dim=1)]
        x = x_frames + x_audio
        y = self.fc(x)
        return y

In [39]:
if (1,2):
    print(1)

1


In [44]:
*[(1, 3), (1, 3)]

SyntaxError: can't use starred expression here (862584758.py, line 1)

In [40]:
x = torch.randn(2, 5, 100, 900)
net = nn.Sequential(
    nn.Conv2d(5, 32, 3, 1, 1),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(32, 64, 3, 1, 1),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(64, 64, 3, 1, 1),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(64, 128, 3, 1, 1),
    nn.MaxPool2d((1, 2), (1, 2)),
    nn.Conv2d(128, 128, 3, 1, 1),
    nn.MaxPool2d((1, 2), (1, 2)),
    nn.Conv2d(128, 256, 3, 1, 1),
    nn.MaxPool2d((1, 2), (1, 2)),
    nn.Conv2d(256, 512, 3, 1, 1),
    nn.MaxPool2d((1, 3), (1, 1)),
    nn.Conv2d(512, 512, 3, 1, 1),
    nn.MaxPool2d(2, 2),
    nn.AdaptiveAvgPool2d((1,1))
    )
fc = nn.Linear(512, 64)

x = net(x)
x = fc(x.squeeze())
x.shape

In [41]:
x.shape

torch.Size([2, 32, 100, 900])

In [54]:
lstm = nn.LSTM(1764, 256, 2, batch_first=True)
lstm2 = nn.LSTM(256, 64, 2, batch_first=True)

out = lstm(torch.randn(2, 5, 1764))[0]
out = lstm2(out)[0]
out.shape


torch.Size([2, 5, 64])

In [8]:
model = RESTM()
x_frames = torch.randn(2, 5, 100, 900)
x_audio = torch.randn(2, 5, 1764)
print(model(x_frames, x_audio).shape)

torch.Size([2, 5, 256])
torch.Size([2, 76])


In [12]:
import torch.nn.functional as F

class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_heads, num_layers):
        super(TransformerModel, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Linear(input_size, hidden_size)
        self.transformer = nn.Transformer(d_model=hidden_size, nhead=num_heads, num_encoder_layers=num_layers)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(1, 0, 2)  # Reshape to (seq_len, batch_size, hidden_size)
        outputs = self.transformer(x)
        x = outputs[0]  # Take the output of the last transformer layer
        x = x.permute(1, 0, 2)  # Reshape back to (batch_size, seq_len, hidden_size)
        x = self.fc(x)
        return x

# Usage example
input_size = 1764  # Input size of the audio data
hidden_size = 256  # Hidden size of the Transformer model
num_heads = 4  # Number of attention heads
num_layers = 4  # Number of transformer layers
num_classes = 10  # Number of output classes

model = TransformerModel(input_size, hidden_size, num_heads, num_layers)
input_data = torch.randn(1, 5, input_size)  # Replace with your own input data
output = model(input_data)

TypeError: forward() missing 1 required positional argument: 'tgt'

In [16]:
32 * 1764

56448

In [20]:
class AudioCNNModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(AudioCNNModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(32 * 1764 // 4, num_classes)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        print(x.shape)
        x = F.max_pool1d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        print(x.shape)
        x = F.max_pool1d(x, kernel_size=2, stride=2)
        print(x.shape)
        x = x.view(x.size(0), -1)
        print(x.shape)
        x = self.fc(x)
        return x

# Usage example
input_channels = 5  # Number of input channels (e.g., mono audio)
num_classes = 76  # Number of output classes

model = AudioCNNModel(input_channels, num_classes)
input_data = torch.randn(64, 5, 1764)  # Replace with your own input data
output = model(input_data)
print(output.shape)

torch.Size([64, 16, 1764])
torch.Size([64, 32, 882])
torch.Size([64, 32, 441])
torch.Size([64, 14112])
torch.Size([64, 76])
